In [32]:
import re

with open("15.input", "r") as f:
    targetY = 2000000
    beaconsOnTargetY = set()
    intervals = []
    for line in f:
        line = line.rstrip()
        if line:
            sensorX, sensorY, beaconX, beaconY = map(int, re.findall(r"=(-?\d+)", line))
            if beaconY == targetY:
                beaconsOnTargetY.add(beaconX)
            distance = abs(sensorX - beaconX) + abs(sensorY - beaconY)
            if sensorY - distance <= targetY <= sensorY + distance:
                # intervalLen = 1 + 2 * (distance - abs(targetY - sensorY))
                halfIntervalLen = distance - abs(targetY - sensorY)
                intervals.append((sensorX - halfIntervalLen, sensorX + halfIntervalLen))
    # sweep line
    endpoints = sorted(
        [(x, i) for itv in intervals for i, x in enumerate(itv)]
        + [
            (x, -1) for x in beaconsOnTargetY
        ]  # -1 to let beacons be processed first when x is the same
    )
    # print(endpoints)
    covered = 0
    beacons = 0  # beacons that are covered by at least one interval
    lastX = None
    result = 0  # length of merged intervals - beacons
    for x, v in endpoints:
        if v == 0:
            covered += 1
            if covered == 1:
                lastX = x
        elif v == 1:
            covered -= 1
            if covered == 0:
                result += x - lastX + 1 - beacons
                beacons = 0
        elif v == -1:
            if covered > 0:
                beacons += 1
    print(result)


4876693


In [31]:
import re

maxCoord = 4000000  # for the example, the maxCoord is 20
minCoord = 0


def checkDistressBeacon(sensorBeaconPair, targetY):
    intervals = []
    for sensorX, sensorY, beaconX, beaconY, distance in sensorBeaconPair:
        if sensorY - distance <= targetY <= sensorY + distance:
            halfIntervalLen = distance - abs(targetY - sensorY)
            intervals.append(
                (
                    max(minCoord, sensorX - halfIntervalLen),
                    min(sensorX + halfIntervalLen, maxCoord),
                )
            )
    endpoints = sorted([(x, i) for itv in intervals for i, x in enumerate(itv)])
    covered = 0
    lastX = None
    result = 0  # number of points that can not have the distress beacon
    coveredIntervals = []
    for x, v in endpoints:
        if v == 0:
            covered += 1
            if covered == 1:
                lastX = x
        elif v == 1:
            covered -= 1
            if covered == 0:
                result += x - lastX + 1 - beacons
                # because there's exactly 1 possible position for the distress beacon
                # if the distress beacon is on line y = targetY,
                #   1. there can be only one interval without either end point minCoord or maxCoord
                #   2. or there can be two intervals, leaving one point in the middle
                if len(coveredIntervals) < 2:
                    coveredIntervals.append((lastX, x))

    # find the x coordinate of the distress beacon on line y = targetY
    if result < maxCoord - minCoord + 1:
        print(coveredIntervals)
        if len(coveredIntervals) == 2:
            print(
                f"Found the distress beacon at ({coveredIntervals[1][0] - 1},{targetY})"
            )
            print(
                "Tuning frequency is", targetY + (coveredIntervals[1][0] - 1) * 4000000
            )
        elif len(coveredIntervals) == 1:
            if coveredIntervals[0][0] == minCoord:
                print(
                    f"Found the distress beacon at ({coveredIntervals[0][1] + 1},{targetY})"
                )
                print(
                    "Tuning frequency is",
                    targetY + (coveredIntervals[0][1] + 1) * 4000000,
                )
            elif coveredIntervals[0][1] == maxCoord:
                print(
                    f"Found the distress beacon at ({coveredIntervals[0][0] - 1},{targetY})"
                )
                print(
                    "Tuning frequency is",
                    targetY + (coveredIntervals[0][0] - 1) * 4000000,
                )
            else:
                print("Error")
        return True
    return False


with open("15.input", "r") as f:
    sensorBeaconPair = []
    for line in f:
        line = line.rstrip()
        if line:
            sensorX, sensorY, beaconX, beaconY = map(int, re.findall(r"=(-?\d+)", line))
            distance = abs(sensorX - beaconX) + abs(sensorY - beaconY)
            sensorBeaconPair.append((sensorX, sensorY, beaconX, beaconY, distance))

    for y in range(minCoord, maxCoord + 1):
        if checkDistressBeacon(sensorBeaconPair, y):
            break


[(0, 2911362), (2911364, 4000000)]
Found the distress beacon at (2911363,2855041)
Tuning frequency is 11645454855041
